## Hands-On: LightGBM

LightGBM is a gradient boosting framework that uses tree-based learning algorithms and is designed for distributed and efficient training, particularly on large datasets. It stands for Light Gradient Boosting Machine and is developed by Microsoft. LightGBM is known for its speed and performance, as well as its ability to handle large-sized data and support GPU learning. It is currently one of the best Supervised Learning algorithms in terms of performance and speed for tabular (structural or dataframe-like) datasets.

**Key Features of LightGBM**

- **Histogram-based Algorithms**: Instead of considering each value for each feature to find the optimal split, it buckets continuous feature values into discrete bins, which speeds up the training process.
- **Gradient-based One-Side Sampling (GOSS)**: Keeps all the instances with large gradients and performs random sampling on the instances with small gradients.
- **Exclusive Feature Bundling (EFB)**: Reduces the number of features by bundling mutually exclusive features, which decreases the training complexity without hurting the model accuracy.

**Advantages of LightGBM**

- Faster training and higher efficiency.
- Lower memory usage.
- Better accuracy with support for parallel and GPU learning.
- Capable of handling large-scale data.

**Disadvantages of LightGBM**

- May overfit on small datasets.
- Requires careful tuning of hyperparameters.

In [29]:
import seaborn as sns
from sklearn.model_selection import GridSearchCV, cross_val_score, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_curve, auc

# Generate predictions on the test set and evaluate the model's performance
# using a confusion matrix, accuracy, precision, recall, and F1 score.

titanic = sns.load_dataset('titanic')
features = ['pclass', 'sex', 'age', 'sibsp', 'parch', 'fare', 'embarked']
target = 'survived'

X = titanic[features]
y = titanic[target]

# Preprocessing for numerical data
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

# Preprocessing for categorical data
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, ['age', 'sibsp', 'parch', 'fare']),
        ('cat', categorical_transformer, ['pclass', 'sex', 'embarked'])
    ])


In [ ]:
import lightgbm as lgb

model = lgb.LGBMClassifier(random_state=0, n_jobs=1, force_row_wise=True)
param_grid = {
    'n_estimators': [50, 100, 200, 300],
    'learning_rate': [0.05, 0.1, 0.2],
    'num_leaves': [15, 31],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0],
}

# Use f1 score as the metrics
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, n_jobs=-1, verbose=2, scoring='f1')

pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                           ('grid_search', grid_search)
                          ])

In [31]:
# Split the data into training and testing sets. 
# Fit the full pipeline (preprocessing and model) on the training data.

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
pipeline.fit(X_train, y_train)

Fitting 5 folds for each of 96 candidates, totalling 480 fits
[LightGBM] [Info] Number of positive: 218, number of negative: 351
[LightGBM] [Info] Total Bins 189
[LightGBM] [Info] Number of data points in the train set: 569, number of used features: 12
[LightGBM] [Info] Number of positive: 218, number of negative: 351
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.383128 -> initscore=-0.476291
[LightGBM] [Info] Start training from score -0.476291
[LightGBM] [Info] Total Bins 192
[LightGBM] [Info] Number of data points in the train set: 569, number of used features: 12
[LightGBM] [Info] Number of positive: 219, number of negative: 351
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.383128 -> initscore=-0.476291
[LightGBM] [Info] Start training from score -0.476291
[LightGBM] [Info] Total Bins 199
[LightGBM] [Info] Number of data points in the train set: 570, number of used features: 12
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.384211 -> initscore=-0.471714
[LightGBM] [Info]

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['age', 'sibsp', 'parch',
                                                   'fare']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['pclass', 'sex',
                                                   'embarked'])])),
                ('grid_search',
                 GridSearchCV(cv=5,
                              estimator=LGBMClassifier(force_row_wise=True,
                                                       n_jobs=1,
                                                       random_state=0),
                              n_jobs=-1,
                              param_grid={'colsample_bytree': [0.8, 1.0],
                                          'learning_rate': [0.05, 0.1, 0.2],
                                          'n_estimators': [50, 100, 200, 300],
                                          'num_leaves': [15, 31],
                                          'subsample': [0.8, 1.0]},
                              scoring='f1', verbose=2))])

In [32]:
best_params = grid_search.best_params_
best_score = grid_search.best_score_

print(f"Best Parameters: {best_params}")
print(f"Best Score: {best_score:.4f}")

Best Parameters: {'colsample_bytree': 0.8, 'learning_rate': 0.05, 'n_estimators': 100, 'num_leaves': 31, 'subsample': 0.8}
Best Score: 0.7668


In [33]:
# Predict class labels for the test set and compute performance metrics (Accuracy, Precision, Recall, F1).
# Also compute ROC curve points and the Area Under the Curve (AUC) using predicted probabilities.

preds = pipeline.predict(X_test)

accuracy = accuracy_score(y_test, preds)
precision = precision_score(y_test, preds)
recall = recall_score(y_test, preds)
f1 = f1_score(y_test, preds)
fpr, tpr, thresholds = roc_curve(y_test, pipeline.predict_proba(X_test)[:,1])
roc_auc = auc(fpr, tpr)

In [34]:
print(f'Accuracy: {accuracy:.2f}')
print(f'Precision: {precision:.2f}')
print(f'Recall: {recall:.2f}')
print(f'F1 Score: {f1:.2f}')
print(f"AUC: {roc_auc:.2f}")

Accuracy: 0.85
Precision: 0.89
Recall: 0.71
F1 Score: 0.79
AUC: 0.90
